# Analyzing Startup Fundraisin Deals from Crunchbase
In this quick data engineering project, some of the techniques will be practiced to get a basic foundation on analyzing startup investments from Crunchbase.com.

## Introduction
Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

That being said, we will use an arbitrary cut off date for the data of October 2013. The dataset can be found here: https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv.

We will mainly work with data engineering and different memory constraints here. For more data analysis or data science type projects, they can be found in the appropriate Github repos. We assume that we only have 10 megabytes of available memory. While crunchbase-investments.csv consumes 10.3 megabytes of disk space, we know that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns), so keep this in mind!

Let's first get familiar with some columns and chunks, using 5000 rows per chunk (this is much less than 10 mb of memory).

In [1]:
# get pandas and set columns to avoid truncating
import pandas as pd
pd.options.display.max_columns = 99

# set the reader to a chunk size of 5000
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

In [2]:
# create my_list
mv_list = []

# for each chunk get the nulls 
for chunk in chunk_iter:
    mv_list.append(chunk.isnull().sum())
    
# add it onto the df and get uniques
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [5]:
# re read
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

# counter for memory footprint
counter = 0

# make a series for memory fp
series_memory_fp = pd.Series()

# for each chunk add onto the series
for chunk in chunk_iter:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1

# drop memory footprint calculation for the index and display
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         4057788
company_name              3591326
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411585
company_city              3505926
investor_permalink        4980548
investor_name             3915666
investor_category_code     622424
investor_country_code     2647292
investor_state_code       2476607
investor_region           3396281
investor_city             2885083
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64

In [6]:
# get the total memory fp in mb
series_memory_fp.sum() / (1024 * 1024)

56.9876070022583

In [7]:
# display previous unique values for analysis
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [10]:
# drop columns representing URL's or containing way too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

# show keep list
keep_cols.tolist

<bound method Index.tolist of Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')>

We did quite a bit already figuring out the memory footprint that the data holsd as well as figuring out which columns are and are not useful for analysis because of nulls. Next we can select some data types...

## Selecting Data Types
Let's first determine which columns shift types across chunks. Note that we only lay the groundwork for this step.

In [11]:
# dict for col types
# key: column name 
# value: List of types
col_types = {}

# re read
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

# for each chunk append col type
for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [12]:
# unique col types dict
uniq_col_types = {}

# for each pair in col types turn it into a set
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
    
# see uniques
uniq_col_types

{'company_category_code': {'object'},
 'company_city': {'object'},
 'company_country_code': {'object'},
 'company_name': {'object'},
 'company_region': {'object'},
 'company_state_code': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'funding_round_type': {'object'},
 'investor_city': {'float64', 'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_name': {'object'},
 'investor_region': {'object'},
 'investor_state_code': {'float64', 'object'},
 'raised_amount_usd': {'float64'}}

In [13]:
# see a few rows of a chunk
chunk.head()

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN


In [14]:
# split columns up into categories
useful_cols = ['company_category_code', 'company_country_code',
               'company_state_code', 'company_region', 'company_city', 
               'investor_name', 'investor_category_code', 
               'investor_country_code', 'investor_state_code',
               'investor_city', 'funding_round_type', 'funded_at',
               'raised_amount_usd']
to_dat_cols = ['funded_at']
to_cat_cols = ['investor_category_code', 'funding_round_type', 
               'investor_region', 'investor_name', 'company_category_code', 
               'investor_state_code', 'company_country_code', 'company_region', 
               'investor_city', 'company_city', 'investor_country_code', 
               'company_state_code']

In [15]:
# cat cols
to_cat_cols_dict = {}

# turn them into cat
for c in to_cat_cols:
    to_cat_cols_dict[c] = 'category'

# float32
to_cat_cols_dict["raised_amount_usd"] = 'float32'

In [16]:
# make function so we don't have to re read
def refined_read_data_in_chunk():
    df_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize=5000, usecols=useful_cols, parse_dates=to_dat_cols, dtype=to_cat_cols_dict)
    return df_iter

In [18]:
# re read using function
df_iter = refined_read_data_in_chunk()

# total mem holder
total_mem = 0

# for each chunk
for i, df in enumerate(df_iter):
    
    # get the mem fp
    mem = df.memory_usage(deep=True).sum()/(1024*1024)
    total_mem += mem
    print("Mem (MB): ", mem)
    
# show the total as well
print("Total Mem (MB): ", total_mem)

# read in a second time for cols
df_iter = refined_read_data_in_chunk()

# show types
for df in df_iter:
    print("\ndtypes: ", df.dtypes)
    print("\ncolumns: ", df.columns.tolist())
    break

Mem (MB):  0.40534114837646484
Mem (MB):  0.31264781951904297
Mem (MB):  0.30964183807373047
Mem (MB):  0.28026294708251953
Mem (MB):  0.3034811019897461
Mem (MB):  0.3014860153198242
Mem (MB):  0.3132619857788086
Mem (MB):  0.3039083480834961
Mem (MB):  0.3341970443725586
Mem (MB):  0.4111642837524414
Mem (MB):  0.2430419921875
Total Mem (MB):  3.518434524536133

dtypes:  company_category_code           category
company_country_code            category
company_state_code              category
company_region                  category
company_city                    category
investor_name                   category
investor_category_code          category
investor_country_code           category
investor_state_code             category
investor_city                   category
funding_round_type              category
funded_at                 datetime64[ns]
raised_amount_usd                float32
dtype: object

columns:  ['company_category_code', 'company_country_code', 'company_state_c

As you can see, even without "normal" analysis, we can do a lot with just looking at data types and memory! Now that we have all of these conversions and optimal changes though, we can begin to use a SQL database...

## Chunks into SQLite
Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

In [19]:
# get sqlite and connect
import sqlite3
conn = sqlite3.connect('crunch.db')

# read in
df_iter = refined_read_data_in_chunk()

# for each chunk append to table (or create first time)
for df in df_iter:
    df.to_sql("crunch_table", conn, if_exists='append', index=False)

# print results of tables
results_df = pd.read_sql('PRAGMA table_info(crunch_table);', conn)
print(results_df)

    cid                    name       type  notnull dflt_value  pk
0     0   company_category_code       TEXT        0       None   0
1     1    company_country_code       TEXT        0       None   0
2     2      company_state_code       TEXT        0       None   0
3     3          company_region       TEXT        0       None   0
4     4            company_city       TEXT        0       None   0
5     5           investor_name       TEXT        0       None   0
6     6  investor_category_code       TEXT        0       None   0
7     7   investor_country_code       TEXT        0       None   0
8     8     investor_state_code       TEXT        0       None   0
9     9           investor_city       TEXT        0       None   0
10   10      funding_round_type       TEXT        0       None   0
11   11               funded_at  TIMESTAMP        0       None   0
12   12       raised_amount_usd       REAL        0       None   0


In [21]:
# we can use this to see the size of the db
!wc --help
!wc -c crunch.db

Usage: wc [OPTION]... [FILE]...
  or:  wc [OPTION]... --files0-from=F
Print newline, word, and byte counts for each FILE, and a total line if
more than one FILE is specified.  With no FILE, or when FILE is -,
read standard input.  A word is a non-zero-length sequence of characters
delimited by white space.
The options below may be used to select which counts are printed, always in
the following order: newline, word, character, byte, maximum line length.
  -c, --bytes            print the byte counts
  -m, --chars            print the character counts
  -l, --lines            print the newline counts
      --files0-from=F    read input from the files specified by
                           NUL-terminated names in file F;
                           If F is - then read names from standard input
  -L, --max-line-length  print the length of the longest line
  -w, --words            print the word counts
      --help     display this help and exit
      --version  output version information 

In [22]:
# convert to mb
print("Total MB for crunch.db:", 5795840/1024**2)

Total MB for crunch.db: 5.52734375


Now that we have everything neatly (as possible) in our db, we can do some inital exploration and analysis on some basic questions as an example...

## Data Exploration and Analysis
We can use the pandas SQLite workflow to analyze startup investments. Also note to remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

This is a data engineering project, but we can still answer some basic example questions such as:
* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

In [23]:
# get a "df" from the db and display
crunch_df = pd.read_sql('select company_category_code, investor_name, raised_amount_usd, funding_round_type from crunch_table;', conn)
print(crunch_df.head())

  company_category_code      investor_name  raised_amount_usd  \
0           advertising  1-800-FLOWERS.COM          2000000.0   
1                  news        10Xelerator            20000.0   
2             messaging        10Xelerator            20000.0   
3              software        10Xelerator            20000.0   
4                   web        10Xelerator            20000.0   

  funding_round_type  
0           series-a  
1              other  
2              other  
3              angel  
4              other  


In [24]:
# sort by amount raised
cmp_type_df = crunch_df.groupby(crunch_df.company_category_code).sum()
cmp_type_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 
print(cmp_type_df.head())

                       raised_amount_usd
company_category_code                   
biotech                     1.103964e+11
software                    7.308452e+10
mobile                      6.477738e+10
cleantech                   5.270523e+10
enterprise                  4.586093e+10


In [25]:
# sort by investor contirbutions
investor_name_df = crunch_df.groupby(crunch_df.investor_name).sum()
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 
print(investor_name_df.head())

                                  raised_amount_usd
investor_name                                      
Kleiner Perkins Caufield & Byers       1.121783e+10
New Enterprise Associates              9.692542e+09
Accel Partners                         6.472126e+09
Goldman Sachs                          6.375459e+09
Sequoia Capital                        6.039402e+09


In [26]:
# sort by investor per startup
investor_name_counts = crunch_df.investor_name.value_counts()
investor_name_df["count"] = investor_name_counts
investor_name_df["avg investment"] = investor_name_df['raised_amount_usd'] / investor_name_df["count"] 
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 
print(investor_name_df.head())

                                  raised_amount_usd  count  avg investment
investor_name                                                             
Kleiner Perkins Caufield & Byers       1.121783e+10    393    2.854409e+07
New Enterprise Associates              9.692542e+09    445    2.178099e+07
Accel Partners                         6.472126e+09    322    2.009977e+07
Goldman Sachs                          6.375459e+09    123    5.183300e+07
Sequoia Capital                        6.039402e+09    369    1.636694e+07


In [27]:
# sort by funding rounds
funding_round_type_counts = crunch_df.funding_round_type.value_counts()
funding_round_type_df = pd.DataFrame()
funding_round_type_df['count'] = funding_round_type_counts
funding_round_type_df.sort_values('count', ascending=False, inplace=True) 

# most popular
print(funding_round_type_df.head())

# least popular
print(funding_round_type_df.tail())

           count
series-a   13938
series-c+  10870
angel       8989
venture     8917
series-b    8794
                count
series-b         8794
other             964
private-equity    357
post-ipo           33
crowdfunding        5


From all of the code displayed above, we can easily make out answers to our questions such as proportions of funds (very lopsided, as expected), which category attracts the most investments (biotech), which investor contributed the most money as well as most oney per startup (both Kleiner Perkins Caufield & Byers), and even which funding rounds were most and least popular (series-a and crowfunding, respectively). Using a SQLite workflow, any analysis inside a db can be just as easy as using a pandas df!

## Further Analysis / Next Steps
Even though all we have left to do now is just upkeeping our db and researching more topics, it really is an open ended question. We can analyze infinite angles, and its usually up to the discretion of the employer. There are a few things to make notes on though for practice.

## Stricter Memory Constraints
Ever try to make things as hard as possible for the fun of it? Of course, this doesn't make sense at all in practice, but you may be left with an outdated machine to work with in a pinch someday, so it's nice to know how it works.

Luckily for us, all this really means is changing up the chunk size! Although this creates more chunks, the robustness of our loops don't really hard code the max number of chunks, so we can play around to see which chunk size would get us to memory footprints of say, under 1 mb.

Even if we were not limited to strict memory contraints, finding the optimal size is important enough anyway that you should always try to find the smallest possible footprint for each chunk that makes sense (you still don't want a million chunks of minimal size).

## Other Crunchbase Datasets
Using the same Github link at the beginning of the project, we can find other datasets on Crunchbase, spanning different times, or even different topics altogether (https://github.com/datahoarder/crunchbase-october-2013).

ALthough most of this analysis will be the same and redundant, some things that we can do are: 
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.
* Use pandas to populate each table in the database, create the appropriate indexes, and so on.

This is ultimately just designing our own database at this point, and more information on this can be found in other projects in the Github repo. Seeing how different datasets can interact with each other in the form of tables is simple yet interesting.

## Conclusion
To sum everything up, we practiced some memory constraints in this project to build up from a basic dataset to a SQLite workflow in order to answer some analysis questions. It turns out that there is a lot more to data than we think!

A lot of the time people overlook the 'boring' aspects of data science, and the subfield of data engineering really brings these topics into the light. Even though they may not be as interesting as building machine learning models, they are still just as important.

Being an expert in both fields may save you one day from an unlikely 